## Main script to perform heart rate estimation of wearable PPG

This script uses both PPG and accelerometer and performs the following steps:
1. Loading all metadata of PPG and IMU
2. Query on data availability + synchronization
3. Loading relevant segment sensor data using tsdf wrapper (start for loop over synchronized segment indices)
4. Synchronize the data (correct indices etc)
5. Data preprocessing
6. Perform pseudo-smoothed Wigner-Ville distribution (PSWVD) on PPG
7. Saving the HR estimates in tsdf format

## Architecture overview
The script implements the following steps:
 - Step 1: PPG preprocessing
 - Step 4: HR estimation


In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from scipy.signal import butter, filtfilt
from pathlib import Path

# Add paths to the toolbox and wrapper
import sys
sys.path.append('../../../../dbpd-toolbox')
sys.path.append('../../../../tsdf4matlab')

import tsdf
import dbpd


## Additional methods

In [3]:
import re

def tsdf_scan_meta(tsdf_data_full_path):
    """
    For each given TSDF directory, transcribe TSDF metadata contents to a list of dictionaries.
    This function is specific for a toolbox data structure mimicking the given MATLAB code.
    
    Parameters:
    - tsdf_data_full_path: Full path to the directory containing TSDF metadata files.
    
    Returns:
    - tsdf: List of dictionaries with metadata from each JSON file in the directory.
    """
    tsdf = []
    
    # Collect all metadata JSON files in the specified directory
    meta_list = list(Path(tsdf_data_full_path).rglob('*_meta.json'))
    for meta_file in meta_list:
        with open(meta_file, 'r') as file:
            json_obj = json.load(file)
            meta_data = {
                'tsdf_meta_fullpath': str(meta_file),
                'subject_id': json_obj['subject_id'],
                'start_iso8601': json_obj['start_iso8601'],
                'end_iso8601': json_obj['end_iso8601']
            }
            tsdf.append(meta_data)
    
    return tsdf

def tsdf_values_idx(metadata_list: list, suffix: str):
    """
    Searches for indices in the metadata list where the file name ends with a specified suffix.
    
    Args:
    - metadata_list (list of names): A list where each item is a dictionary containing metadata,
      including a 'file_name' key with the file's name as its value.
    - suffix (str): The suffix to search for in the file names. It should include the extension, 
      such as '.bin'.
    
    Returns:
    - list: A list of file names where the file names end with the specified suffix.
    """
    # Compile a regular expression pattern to match file names ending with the given suffix
    pattern = re.compile(rf".*{re.escape(suffix)}$")
    for metadata in metadata_list:
        # Check if the 'file_name' in metadata matches the pattern
        if pattern.search(metadata):
            return metadata
    return None

In [14]:
# Constants
UNIX_TICKS_MS = 1000.0
FS_PPG = 30

# Paths
raw_data_root = '../../../tests/data/1.sensor_data/'
ppp_data_path_ppg = os.path.join(raw_data_root, 'PPG')
meta_ppg = tsdf_scan_meta(ppp_data_path_ppg)

sqa_data_path = '../../../tests/data/4.predictions/ppg'
sqa_output_list = list(Path(sqa_data_path).glob('*_meta.json'))
meta_path_sqa = sqa_output_list[0]


# Load PPG data
metadata_dict = tsdf.load_metadata_from_path(meta_path_sqa)

metadata_binaries_list = metadata_dict.keys()

# sync_idx = tsdf_values_idx(metadata_binaries_list, 'sync')
sync_idx = "classification_sqa_sync.bin"
data_sync_np:np.ndarray = tsdf.load_ndarray_from_binary(metadata_dict[sync_idx])


# TODO: Fix the following code to work
data_sync_np = data_sync_np[~(data_sync_np == 0).all(axis=1)]
n_segments_sync = data_sync_np.shape[0]

# Load classification data
# ppg_prob_idx = tsdf_values_idx(metadata_binaries_list, 'ppg')
ppg_prob_idx = "classification_sqa_ppg.bin"
ppg_post_prob_np:np.ndarray = tsdf.load_ndarray_from_binary(metadata_dict[ppg_prob_idx])
# imu_idx = tsdf_values_idx(metadata_binaries_list, 'sqa_imu')
imu_idx = "classification_sqa_imu.bin"
imu_label_np:np.ndarray = tsdf.load_ndarray_from_binary(metadata_dict[imu_idx])

# Initialize the indices array for classification epochs
print(data_sync_np.shape)
start_end_indices = np.zeros((len(data_sync_np[:, 3]), 2), dtype=int)
for i in range(len(data_sync_np[:, 3])):
    if i == 0:
        start_end_indices[i, 0] = 1
    else:
        start_end_indices[i, 0] = start_end_indices[i-1, 1] + 1
    start_end_indices[i, 1] = np.sum(data_sync_np[:i+1, 3])

(1, 4)


In [15]:
# Heart Rate Estimation Parameters
MIN_WINDOW_LENGTH = 10
MIN_HR_SAMPLES = MIN_WINDOW_LENGTH * FS_PPG
THRESHOLD_SQA = 0.5
HR_EST_LENGTH = 2
HR_EST_SAMPLES = HR_EST_LENGTH * FS_PPG

# Time-frequency Distribution Parameters
TFD_LENGTH = 10
KERN_TYPE = 'sep'  # Placeholder type
WIN_TYPE_DOPPLER = 'hamm'
WIN_TYPE_LAG = 'hamm'
WIN_LENGTH_DOPPLER = 1
WIN_LENGTH_LAG = 8
DOPPLER_SAMPLES = FS_PPG * WIN_LENGTH_DOPPLER
LAG_SAMPLES = WIN_LENGTH_LAG * FS_PPG
KERN_PARAMS = {'doppler_samples': DOPPLER_SAMPLES, 'win_type_doppler': WIN_TYPE_DOPPLER, 'lag_samples': LAG_SAMPLES, 'win_type_lag': WIN_TYPE_LAG}

# Placeholder for moving average filter
MA = {
    'value': 1,
    'window': 30,
    'FC': np.ones(30) / 30  # Filter coefficients
}

v_hr_ppg = []
t_hr_unix = []

In [16]:
# Main Loop over all synchronized segments
for n in range(n_segments_sync):
    ppg_indices = data_sync_np[n, :2]
    ppg_segment = data_sync_np[n, 2]

    class_start = start_end_indices[n, 0]
    class_end = start_end_indices[n, 1]

    meta_path_ppg = meta_ppg[ppg_segment]['tsdf_meta_fullpath']
    # TODO: Fix loading tsdf
    metadata_list_ppg, data_list_ppg = tsdf.load_tsdf_metadata_from_path(meta_path_ppg)

    time_idx_ppg = tsdf.get_index(metadata_list_ppg, 'time')
    values_idx_ppg = tsdf.get_index(metadata_list_ppg, 'samples')

    t_iso_ppg = metadata_list_ppg[time_idx_ppg]['start_iso8601']
    datetime_ppg = datetime.strptime(t_iso_ppg, '%d-%b-%Y %H:%M:%S %Z').replace(tzinfo=datetime.timezone.utc)
    ts_ppg = datetime_ppg.timestamp() * UNIX_TICKS_MS

    t_ppg = np.cumsum(data_list_ppg[time_idx_ppg]) + ts_ppg
    tr_ppg = (t_ppg - ts_ppg) / UNIX_TICKS_MS

    v_ppg = data_list_ppg[values_idx_ppg]

    v_ppg = v_ppg[ppg_indices[0]:ppg_indices[1]]
    tr_ppg = tr_ppg[ppg_indices[0]:ppg_indices[1]]

    ts_sync = ts_ppg + tr_ppg[0] * UNIX_TICKS_MS
    tr_ppg -= tr_ppg[0]

    fs_ppg_est = 1 / np.median(np.diff(tr_ppg))

    if len(v_ppg) < FS_PPG * MIN_WINDOW_LENGTH:
        print('Sample is of insufficient length!')
        continue

    # Placeholder functions
    v_ppg_pre, tr_ppg_pre = preprocessing_ppg(tr_ppg, v_ppg, FS_PPG)
    
    class_ppg_segment = ppg_post_prob_np[class_start:class_end]
    class_acc_segment = imu_label_np[class_start:class_end]

    # Assign window-level probabilities to individual samples
    data_prob_sample = sample_prob_final(class_ppg_segment, class_acc_segment, FS_PPG)

    sqa_label = np.zeros(len(data_prob_sample))
    sqa_label[data_prob_sample > THRESHOLD_SQA] = 1

    v_start_idx, v_end_idx = extract_hr_segments(sqa_label, MIN_HR_SAMPLES)

    for i in range(len(v_start_idx)):
        if v_start_idx[i] < 2 * FS_PPG or v_end_idx[i] > len(v_ppg_pre) - 2 * FS_PPG:
            continue
        
        rel_ppg = v_ppg_pre[v_start_idx[i]:v_end_idx[i]]
        rel_time = tr_ppg_pre[v_start_idx[i]:v_end_idx[i]]

        rel_ppg_spwvd = v_ppg_pre[v_start_idx[i] - FS_PPG*2:v_end_idx[i] + FS_PPG*2]
        hr_est = PPG_TFD_HR(rel_ppg_spwvd, TFD_LENGTH, MA, FS_PPG, KERN_TYPE, KERN_PARAMS)

        if len(rel_ppg) % 60 != 0:
            hr_time = rel_time[0:len(rel_ppg) - FS_PPG:HR_EST_SAMPLES]
        else:
            hr_time = rel_time[0:len(rel_ppg):HR_EST_SAMPLES]

        t_epoch_unix = hr_time * UNIX_TICKS_MS + ts_sync
        v_hr_ppg.append(hr_est)
        t_hr_unix.append(t_epoch_unix)

IndexError: list index out of range

In [ ]:
# Save the HR output in TSDF format
data_hr_est = [np.array(t_hr_unix) / UNIX_TICKS_MS, np.array(v_hr_ppg)]

location = "../../../tests/data/5.quantification/ppg"
os.makedirs(location, exist_ok=True)

if not t_hr_unix:
    start_time_iso = datetime.fromtimestamp(0, tz=datetime.timezone.utc).strftime('%d-%b-%Y %H:%M:%S %Z')
    end_time_iso = datetime.fromtimestamp(0, tz=datetime.timezone.utc).strftime('%d-%b-%Y %H:%M:%S %Z')
else:
    start_time_iso = datetime.fromtimestamp(t_hr_unix[0] / UNIX_TICKS_MS, tz=datetime.timezone.utc).strftime('%d-%b-%Y %H:%M:%S %Z')
    end_time_iso = datetime.fromtimestamp(t_hr_unix[-1] / UNIX_TICKS_MS, tz=datetime.timezone.utc).strftime('%d-%b-%Y %H:%M:%S %Z')

metafile_pre_template = metadata_binaries_list[time_idx_ppg]
metafile_pre_template['start_iso8601'] = start_time_iso
metafile_pre_template['end_iso8601'] = end_time_iso

metafile_time = metafile_pre_template.copy()
metafile_values_hr = metafile_pre_template.copy()

metafile_time['channels'] = ['time']
metafile_time['units'] = ['time_absolute_unix_s']
metafile_time['file_name'] = 'hr_est_time.bin'

metafile_values_hr['channels'] = ['HR estimates']
metafile_values_hr['units'] = ['min^-1']
metafile_values_hr['freq_sampling_original'] = round(fs_ppg_est, 2)
metafile_values_hr['file_name'] = 'hr_est_values.bin'

meta_class = [metafile_time, metafile_values_hr]

mat_metadata_file_name = "hr_est_meta.json"
tsdf.save_tsdf_data(meta_class, data_hr_est, location, mat_metadata_file_name)
